In [1]:
#@title Data retrieval
import os, requests, pdb

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)

In [2]:
#@title Import matplotlib and set defaults
from matplotlib import rcParams 
from matplotlib import pyplot as plt

rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [3]:
#@title Data loading
import numpy as np

alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

In [ ]:
region_colors = ['lightblue', 'gray', 'violet']
brain_areas = [["MOp", "MOs","PL"], # frontal choice areas
                ["SCm","ZI"], # midbrain choice areas
                ["CP","SNr"], # basal ganglia 
               ]
# Assign each area an index
area_to_index = dict(root=0)
counter = 1
for group in brain_groups:
    for area in group:
        area_to_index[area] = counter
        counter += 1

# Figure out which areas are in each dataset
areas_by_dataset = np.zeros((counter, len(alldat)), dtype=bool)
for j, d in enumerate(alldat):
    for area in np.unique(d['brain_area']):
        i = area_to_index[area]
        areas_by_dataset[i, j] = True


    

In [4]:
def region_finder(region):
    # Find sessions with areas of interest
    session_idx = []
    neuron_idx = []
    for i in range(39):
        dat = alldat[i]
        areas = dat['brain_area']
        decision = areas==region
        if any(decision):
            n = [j for j, x in enumerate(decision) if x]
            session_idx.append(i)
            neuron_idx.append(n)
            #pdb.set_trace()
    return session_idx, neuron_idx

In [5]:
region_finder("MOs")


([0, 3, 4, 7, 11, 12, 13, 21, 24, 25, 26, 29, 30, 31, 34, 35, 36, 38],
 [[1,
   4,
   7,
   8,
   12,
   17,
   19,
   22,
   23,
   25,
   26,
   27,
   29,
   30,
   32,
   33,
   34,
   38,
   49,
   50,
   52,
   53,
   55,
   56,
   60,
   64,
   65,
   70,
   71,
   79,
   84,
   85,
   86,
   88,
   93,
   100,
   101,
   102,
   107,
   109,
   110,
   111,
   113,
   121,
   123,
   127,
   130,
   132,
   136,
   137,
   138,
   143,
   144,
   145,
   149,
   156,
   159,
   161,
   162,
   163,
   164,
   167,
   172,
   175,
   176,
   177,
   180,
   188,
   190,
   191,
   195,
   210,
   214,
   223,
   226,
   227,
   229,
   235,
   239,
   240,
   242,
   246,
   247,
   248,
   249,
   255,
   257,
   259,
   261,
   268,
   269,
   278,
   286,
   297,
   298,
   301,
   303,
   308,
   318,
   323,
   326,
   330,
   333,
   334,
   335,
   341,
   345,
   348,
   349,
   353,
   354,
   358,
   360],
  [947,
   954,
   958,
   971,
   979,
   982,
   983,
   985,

In [8]:
# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx. 
session_idx, neuron_idx = region_finder("MOs")
dat = alldat[session_idx[1]]
print(dat.keys())

dt = dat['bin_size'] # binning at 10 ms
zebras = dat['spks']
zebras = zebras[neuron_idx[1]]
NT = zebras.shape[-1]
ax = plt.subplot(1,5,1)
response = dat['response'] # right - nogo - left (-1, 0, 1)
vis_right = dat['contrast_right'] # 0 - low - high
vis_left = dat['contrast_left'] # 0 - low - high
plt.plot(dt * np.arange(NT), 1/dt * dat['spks'][:,response>=0].mean(axis=(0,1))) # left responses
plt.plot(dt * np.arange(NT), 1/dt * dat['spks'][:,response<0].mean(axis=(0,1))) # right responses
plt.plot(dt * np.arange(NT), 1/dt * dat['spks'][:,vis_right>0].mean(axis=(0,1))) # stimulus on the right
plt.plot(dt * np.arange(NT), 1/dt * dat['spks'][:,vis_right==0].mean(axis=(0,1))) # no stimulus on the right

plt.legend(['left resp', 'right resp', 'right stim', 'no right stim'], fontsize=12)
ax.set(xlabel  = 'time (sec)', ylabel = 'firing rate (Hz)');


dict_keys(['spks', 'wheel', 'pupil', 'response', 'response_time', 'bin_size', 'stim_onset', 'contrast_right', 'contrast_left', 'brain_area', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'active_trials', 'contrast_left_passive', 'contrast_right_passive', 'spks_passive', 'pupil_passive', 'wheel_passive', 'prev_reward', 'ccf', 'ccf_axes', 'cellid_orig', 'reaction_time', 'face', 'face_passive', 'licks', 'licks_passive'])
--Return--
None
> <ipython-input-8-3b8fb16d2ce1>(10)<module>()
      8 zebras = zebras[neuron_idx[1]]
      9 NT = zebras.shape[-1]
---> 10 pdb.set_trace()
     11 ax = plt.subplot(1,5,1)
     12 response = dat['response'] # right - nogo - left (-1, 0, 1)

ipdb> dt
0.01
ipdb> NT
250
ipdb> zebras
array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0

BdbQuit: 